In [ ]:
# Load Pictures function
def LoadImages(path):
    import csv
    
    File = []
    with open(path) as csvfile:
        inputfile = csv.reader(csvfile)
        for row in inputfile:
            File.append(row)
            
    return File

In [ ]:
# Load sensogram function

def LoadSensogram():
    import csv
    
    # Initialize the testmatrix array
    testmatrix = []
    # Initialize dummy variable
    num = 0
    # Open text file
    with open('sensogram.txt') as csvfile:
        inputfile = csv.reader(csvfile)
        for row in inputfile:
            testmatrix.append(row)
            for i in range(len(row)):
                testmatrix[num][i] = int(row[i])
            num = num+1    
        # At this point we get all values in packets of data being integer, good job, had to loop through each array
        # as it comes in to convert it from a string to an integer.
    return testmatrix

In [ ]:
def DisplayImages(x,test_array,picmatrix,n):

    import tkinter as tk
    from PIL import Image, ImageTk

    master = tk.Tk()
    
    # This line of code specifies where the window opens up and how big it is
    master.geometry("1900x500+300+300")
    v = tk.IntVar()
#     v.set(1)

    tk.Label(master, 
        text="""Click over the image which the pattern matches most with:""",
        justify = tk.LEFT,
        padx = 20)#.pack() 
    
    def ShowChoice():
#         print(v.get())
        master.destroy()
        selection = v.get()
        return selection
    
    img = []
    photo = []
    
#     f1 = tk.Frame(master)
#     f1.grid(row=2, column=5, sticky="nsew")
    
    for i in range(n):
        img.append(Image.open(','.join(picmatrix[test_array[i]])))
        img[i] = img[i].resize((250,250),Image.NEAREST)
        photo.append(ImageTk.PhotoImage(img[i]))

        but = tk.Radiobutton(master, 
              text="Image",image=photo[i],
              padx = 20, 
              variable=v,indicatoron = 0,command=ShowChoice, 
              value=i)#.pack(side)#=tk.LEFT)
        
        if i < 5:
            but.grid(row=0,column=i)
        else:
            but.grid(row=1,column=i-5)

#         pack(anchor=tk.W)  
#         But.grid(row=0, column=i)
    
    master.mainloop()
    return v.get()

In [ ]:
def DisplayChinese(x,ChineseCharac,picmatrix,time):

    import tkinter as tk
    from PIL import Image, ImageTk

    master = tk.Tk()
    # This line closes the window after 2000ms have passed
    master.after(time, lambda: master.destroy())
    
    # This line of code specifies where the window opens up and how big it is
    master.geometry("600x500+100+100")
    v = tk.IntVar()
#     v.set(1)

    img = []
    photo = []
    
    img.append(Image.open(','.join(picmatrix[x])))
    img[0] = img[0].resize((250,250),Image.NEAREST)
    photo.append(ImageTk.PhotoImage(img[0]))
    img.append(Image.open(','.join(ChineseCharac[x])))
    img[1] = img[1].resize((250,250),Image.NEAREST)
    photo.append(ImageTk.PhotoImage(img[1]))

    tk.Radiobutton(master, 
              text="Image",image=photo[0],
              padx = 20, 
              variable=v,indicatoron = 0,
              value=1).pack(side=tk.LEFT)
    tk.Radiobutton(master, 
              text="Chinese",image=photo[1],
              padx = 20, 
              variable=v,indicatoron = 0,
              value=2).pack(side=tk.LEFT)
#         pack(anchor=tk.W)  
#         But.grid(row=0, column=i)

    master.mainloop()
    return master
# import time
# picmatrix = LoadPictures()
# for i in range(3):
#     DisplayChinese(i,picmatrix,picmatrix)
#     time.sleep(1)

In [ ]:
def ActivArray(testmatrix,i):    
    # Initialize array to store testmatrix values
    activ_array = []        
    # Loop through the possible number of examples (7 is the possible nodes)
    for q in range(7):
        if (testmatrix[i][testmatrix[i][q+7]-1] == 1): # This checks if the current node is supposed to be on/off
            activ_array.append(testmatrix[i][q+7]+4) # Add position of the node to array
            activ_array.append((testmatrix[i][q+14]//3)+70) # Add delay of the node to array, THIS IS FOR STRENGTH
        # Print the array of data we are sending
#     print(activ_array)    
    return activ_array

In [ ]:
# This is the function to emit to Serial but without display the pictures

def TestPattern(ser,testmatrix,i):
    import struct
    import time
    
    # Read the incoming line
    print(ser.readline())
    
    while(True):
    
    # If the incoming line matches the trigger
        if (ser.readline() == b'ready\r\n'):
#     if ser == 'hi':
        
        # Then send flag to Arduino to start listening
            ser.write(b'1') # Need to include the b so that data can be sent
                        # this is python 3 syntax
    
        # Get the current array for the pattern
            activ_array = ActivArray(testmatrix,i)
        # Read length of the array
            Counter = len(activ_array)
    
        # Send length of the array to Arduino so it knows how many to wait for
            ser.write(struct.pack('>B', Counter)) # Convert the integers to bits to send via Serial
            print(struct.pack('>B', Counter))
        # Initialize dummy variable to loop through array
            c = 0
        # Loop through length of array sending values
            for y in range(Counter//2):
                ser.write(struct.pack('>B', activ_array[c]))
                ser.write(struct.pack('>B', activ_array[c+1]))
#             print('Im sending %d %d' % (activ_array[c], activ_array[c+1]))
            # Jump two to go to the next pair
                c = c+2
        # Define waiting length
#         time.sleep(1)
            break
        else:
            time.sleep(1)
    return 1

In [ ]:
# This is the Testing Function and Storing values into the Results Matrix

def TestFunction(x,picmatrix,order,n,t):
    import time

# So I'm allowing the user to select between 1-5, because of the way they are pasted, I know which
# picture corresponds to each number. Knowing the picture name, I can correlate to the picture that 
# should have appeared with the pattern by comparing the names I guess. 

    # Generate the random array, choose out of 20 and pick 4
    test_array = PickRandomIndexes(x,order,t)

    selection = DisplayImages(x,test_array,picmatrix,n)

    time.sleep(1)

    TestforthePicture(test_array,x,selection)
    
    return test_array, selection

In [ ]:
#  This portion asks the user to select which picture is the one they recognize and saves the data

def TestforthePicture(test_array,num,selection):
    
    if num == test_array[selection]:
#         print('hooray')
        results.append(1)
    else:
#         print('not yet')
        results.append(0)
        
    return results

In [ ]:
def TestChineseCharacters(x,ChineseCharac):
#     img = Image.open(','.join(ChineseCharac[x]))
#     return img.show()

    import tkinter as tk
    from PIL import Image, ImageTk

    master = tk.Tk()
    # This line closes the window after 2000ms have passed
    master.after(2000, lambda: master.destroy())
    
    # This line of code specifies where the window opens up and how big it is
    master.geometry("600x500+0+0")
    v = tk.IntVar()
#     v.set(1)

    img = []
    photo = []
    
    img.append(Image.open(','.join(ChineseCharac[x])))
    img[0] = img[0].resize((250,250),Image.NEAREST)
    photo.append(ImageTk.PhotoImage(img[0]))

    tk.Radiobutton(master, 
              text="Chinese",image=photo[0],
              padx = 20, 
              variable=v,indicatoron = 0,
              value=2).pack(side=tk.LEFT)
#         pack(anchor=tk.W)  
#         But.grid(row=0, column=i)

    master.mainloop()
    return

In [ ]:
def PickRandomIndexes(x,order,z):
    
    test_array = []
    test_array.append(x)
    a = 0
    length = len(order)-1
    while a < z:
        # This code is only for the spanish test, where for the test I only want the same
        # vowels being presented, otherwise advantage on learning
        randy = random.randint(0,length)
        a = a+1
#         if randy in test_array:
        if order[randy] in test_array:
#             print('one down')
            a = a-1
        else:
            test_array.append(order[randy])
            
    random.shuffle(test_array)   
#     print(test_array)
    return test_array

In [ ]:
def DicPickRandomIndexes(x,order,z):
    
    def get_digit(number, n):
        return number // 10**n % 10
    
    test_array = []
    test_array.append(x)
    a = 0
    b = 0
    length = len(order)-1
    while a < z:
        # This code is only for the spanish test, where for the test I only want the same
        # vowels being presented, otherwise advantage on learning
        order_min = min(order)
        order_max = max(order)

        number = get_digit(x,0)
        if number > 5:
            number = number-5
#         print('this is the number: %d' %number)
        
        order_min_num = get_digit(order_min,0)
        add_num = number - order_min_num
        new_min = order_min + add_num
        randy = new_min + b
        
        ####  ------------ ####
#         randy = random.randint(0,length)
        a = a+1
        if randy in test_array:
#         if order[randy] in test_array:
#             print('one down')
            a = a-1
            b = b+5
        else:
#             test_array.append(order[randy])
            test_array.append(randy)
            
    random.shuffle(test_array)   
            
    return test_array

In [ ]:
def DetailedInfo(results,order,x):
    if x == 1:
        for i in range(len(order)):
            activ_array = ActivArray(testmatrix,order[i])
            print(results[i],picmatrix[order[i]],len(activ_array))
    elif x == 0:
        for i in range(len(order)):
            print(results[i],picmatrix[order[i]])
    return
# DetailedInfo(results,order)

In [ ]:
def PrintScore(results):
    print('%d correct out of %d ' % ((sum(results),len(results))))
    return